In [1]:
!pip install tensorflow gradio pandas scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import gradio as gr

In [3]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

In [4]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),  # Input layer (4 features)
    Dropout(0.2),  # Prevent overfitting
    Dense(32, activation='relu'),  # Hidden layer
    Dropout(0.2),
    Dense(3, activation='softmax')  # Output layer (3 classes)
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.3999 - loss: 1.0415 - val_accuracy: 0.8667 - val_loss: 0.8377
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6931 - loss: 0.8240 - val_accuracy: 0.9000 - val_loss: 0.6745
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7846 - loss: 0.7179 - val_accuracy: 0.9000 - val_loss: 0.5617
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7824 - loss: 0.6240 - val_accuracy: 0.9000 - val_loss: 0.4858
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7501 - loss: 0.5580 - val_accuracy: 0.9000 - val_loss: 0.4286
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8319 - loss: 0.4738 - val_accuracy: 0.9000 - val_loss: 0.3838
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7516 - loss: 0.4949 - val_accuracy: 0.9000 - val_loss: 0.3530
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8449 - loss: 0.3921 - val_accuracy: 0.9000 - val_loss: 0.3273
Epoch 9/100

In [5]:
# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 100.00%


In [6]:
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
    try:
        # Convert inputs to array and scale
        input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
        input_scaled = scaler.transform(input_data)

        # Predict
        prediction = model.predict(input_scaled)
        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction) * 100

        return {
            "Predicted Class": iris.target_names[predicted_class],
            "Confidence (%)": f"{confidence:.2f}%",
            "Class Probabilities": {
                iris.target_names[0]: f"{prediction[0][0]*100:.2f}%",
                iris.target_names[1]: f"{prediction[0][1]*100:.2f}%",
                iris.target_names[2]: f"{prediction[0][2]*100:.2f}%"
            }
        }
    except Exception as e:
        return {"Error": str(e)}

# Define Gradio interface
inputs = [
    gr.Slider(4.0, 8.0, step=0.1, value=5.8, label="Sepal Length (cm)"),
    gr.Slider(2.0, 4.5, step=0.1, value=3.0, label="Sepal Width (cm)"),
    gr.Slider(1.0, 7.0, step=0.1, value=4.0, label="Petal Length (cm)"),
    gr.Slider(0.1, 2.5, step=0.1, value=1.2, label="Petal Width (cm)")
]

outputs = [
    gr.Label(label="Predicted Class"),
    gr.Label(label="Confidence"),
    gr.JSON(label="Class Probabilities")
]

interface = gr.Interface(
    fn=predict_iris,
    inputs=inputs,
    outputs=outputs,
    title="🌼 Iris Flower ANN Classifier",
    description="Predict iris species using an Artificial Neural Network"
)

# Launch the interface
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f9da40c1d7922e34a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
!pip install tensorflow

In [8]:
input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])  # Shape (1, 4)

NameError: name 'sepal_length' is not defined

In [9]:
# Install dependencies
!pip install tensorflow gradio numpy scikit-learn

# Imports
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import gradio as gr

# Load and preprocess data
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

# Build ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=0)

# Gradio interface
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
    try:
        input_data = np.array([[float(sepal_length), float(sepal_width), float(petal_length), float(petal_width)]])
        input_scaled = scaler.transform(input_data)
        prediction = model.predict(input_scaled, verbose=0)
        class_idx = np.argmax(prediction)
        return {
            "Predicted Class": iris.target_names[class_idx],
            "Confidence (%)": f"{np.max(prediction)*100:.2f}%"
        }
    except Exception as e:
        return {"Error": str(e)}

iface = gr.Interface(
    fn=predict_iris,
    inputs=[
        gr.Slider(4.0, 8.0, value=5.8, label="Sepal Length (cm)"),
        gr.Slider(2.0, 4.5, value=3.0, label="Sepal Width (cm)"),
        gr.Slider(1.0, 7.0, value=4.0, label="Petal Length (cm)"),
        gr.Slider(0.1, 2.5, value=1.2, label="Petal Width (cm)")
    ],
    outputs="json",
    title="🌼 Iris ANN Classifier"
)

iface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cc8fce4cac1360c2a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
